In [1]:
import pandas as pd

In [2]:
DATA_PATH = 'data/world.xlsx'

In [3]:
import urllib.request

url = 'https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide.xlsx'
urllib.request.urlretrieve(url, DATA_PATH)

('data/world.xlsx', <http.client.HTTPMessage at 0x7f81d0105cc0>)

In [40]:
df = pd.read_excel(DATA_PATH)
df = df.rename(columns={
    'dateRep': 'date',
    'countriesAndTerritories': 'country',
    'popData2018': 'population'
})
del df['countryterritoryCode']

df.loc[df['country'] == 'Namibia', 'geoId'] = 'NAMIBIA'

df.loc[df['country'] == 'Czechia', 'population'] = 10650000
df['population'] = df['population'].fillna(0)

assert all(df.count()['date'] == x for x in df.count())

print(df.shape)
df.head()

(9717, 9)


,date,day,month,year,cases,deaths,country,geoId,population
0,2020-04-09,9,4,2020,56,3,Afghanistan,AF,37172386.0
1,2020-04-08,8,4,2020,30,4,Afghanistan,AF,37172386.0
2,2020-04-07,7,4,2020,38,0,Afghanistan,AF,37172386.0
3,2020-04-06,6,4,2020,29,2,Afghanistan,AF,37172386.0
4,2020-04-05,5,4,2020,35,1,Afghanistan,AF,37172386.0


In [42]:
def _apply(g):
    g = g.sort_values(by='date', ascending=False)
    
    g['tot_deaths'] = g['deaths'][::-1].cumsum()[::-1]
    g['tot_cases'] = g['cases'][::-1].cumsum()[::-1]
    
    return g

df = df.groupby('country').apply(_apply)
df.index = range(len(df))
df

,date,day,month,year,cases,deaths,country,geoId,population,tot_deaths,tot_cases
0,2020-04-09,9,4,2020,56,3,Afghanistan,AF,37172386.0,14,423
1,2020-04-08,8,4,2020,30,4,Afghanistan,AF,37172386.0,11,367
2,2020-04-07,7,4,2020,38,0,Afghanistan,AF,37172386.0,7,337
3,2020-04-06,6,4,2020,29,2,Afghanistan,AF,37172386.0,7,299
4,2020-04-05,5,4,2020,35,1,Afghanistan,AF,37172386.0,5,270
...,...,...,...,...,...,...,...,...,...,...,...
9712,2020-03-25,25,3,2020,0,0,Zimbabwe,ZW,14439018.0,1,2
9713,2020-03-24,24,3,2020,0,1,Zimbabwe,ZW,14439018.0,1,2
9714,2020-03-23,23,3,2020,0,0,Zimbabwe,ZW,14439018.0,0,2
9715,2020-03-22,22,3,2020,1,0,Zimbabwe,ZW,14439018.0,0,2


In [43]:
df.to_csv('vis/src/data.csv')